In [ ]:
## 找到所有token pair 的次数


def get_stats(tokens):
    counts = {}
    for pairs in zip(tokens,tokens[1:]):
        counts[pairs] = counts.get(pairs, 0 ) + 1 
    return counts     

stats = get_stats(tokens)

print(stats)
# pair = (82,101)

# decoded = ''.join(chr(b) for b in pair)

# print(decoded)

# sorted_dict = sorted(((v,k) for k,v in stats.items()), reverse=True)

# print(sorted_dict)
    

In [ ]:
top_pairs = max(stats, key=stats.get)

print(top_pairs)

In [ ]:
print(stats.get) 

In [ ]:
def merge(tokens,pair,new_idx):
    new_tokens = []
    
    i = 0 
    
    while i < len(tokens):
        if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i+1] == pair[1]:
            new_tokens.append(new_idx)
            i += 2 
        else:
            new_tokens.append(tokens[i])
            i+=1
            
    return new_tokens

# tokens = [1,2,3,3,4,5,6,7,3,3]
pair = (3,3)
idx = 8

new_tokens = merge(tokens,top_pairs,idx)

print(new_tokens)
print(len(new_tokens))

In [ ]:
from datasets import load_dataset

dataset = load_dataset("roneneldan/TinyStories")



In [ ]:
dataset

In [ ]:
text = dataset["train"]["text"][:100]

text = "/n".join(text)

text = text.encode("utf-8")
text = list(map(int,text))
print(len(text))

In [ ]:
def train_bpe(text,vocab_size=300):
    ## 1. encoding text to utf-8 tokens
    ## 2. find the pair with max amount, calculate the occur times of each pairs
    ## 3. merge 300 -256 times 
    
    def get_token(text):
        text = text.encode("utf-8")
        text = list(map(int,text))
        print(len(text))
        return text
    
    def get_stats(text):
        counts = {}
        
        for pair in zip(text,text[1:]):
            counts[pair] = counts.get(pair,0) + 1
        return counts

    def find_max_pair(counts):
        pair = max(counts, key=counts.get)
        print(pair)
        
        return pair
    
    def merge(text,merge_pair,merge_idx):
        i = 0
        new_stat = []
        while i < len(text):
            if i < len(text) -1 and text[i]==merge_pair[0] and text[i+1]==merge_pair[1]:
                new_stat.append(merge_idx)
                i+=2
            else:
                new_stat.append(text[i])
                i+=1
        return new_stat
              
                
    text = get_token(text)
    merge_time = vocab_size - 256
    
    merges = {}
    for i in range(merge_time):
        counts = get_stats(text)
        max_pair = find_max_pair(counts)
        idx = 256 + i
        print(f"merging {max_pair} into new index {idx}")
        text = merge(text, max_pair, idx)
        merges[max_pair] = idx
        
    return merges
           

    
from datasets import load_dataset

dataset = load_dataset("roneneldan/TinyStories")

dataset = dataset["train"]["text"][:100]

print(dataset)

text = "\n".join(dataset)
merges = train_bpe(text,vocab_size=400)

print(merges)
    
    



In [ ]:
vocab = {i: bytes([i]) for i in range(256)}

for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    return b"".join(vocab[i] for i in ids).decode("utf-8", errors="replace")
print(decode([122]))

In [ ]:
## decode 阶段，我们需要创建一个vocab，那就要先把原始的256 个创建进去

vocab = {i : bytes([i]) for i in range(256)}


##merges {(p0,p1):int}
for (p0,p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
    
def decode(input_ids):
    
    text = b"".join(vocab[i] for i in input_ids).decode("utf-8", errors="replace")
    
    return text
def encode(text):
    ids = []
    
    text.encode()
decode([299,97])

In [ ]:
def encode(text, merges):
    """
    text: str
    merges: dict[(int, int) -> int]
    return: list[int]  (token ids)
    """

    # 1. text -> byte tokens (0–255)
    tokens = list(text.encode("utf-8"))

    # 2. repeatedly apply BPE merges
    while True:
        # find all mergeable pairs in current tokens
        candidates = {}
        for pair in zip(tokens, tokens[1:]):
            if pair in merges:
                candidates[pair] = merges[pair]

        # no mergeable pair left
        if not candidates:
            break

        # choose the earliest learned merge
        pair = min(candidates, key=lambda p: candidates[p])

        new_tokens = []
        i = 0
        while i < len(tokens):
            if i < len(tokens) - 1 and (tokens[i], tokens[i+1]) == pair:
                new_tokens.append(merges[pair])
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1

        tokens = new_tokens

    return tokens

In [ ]:
encode("asdadhaiufaif",merges)

In [ ]:
decode([97, 115, 310, 100, 276, 105, 117, 102, 97, 105, 102])

In [ ]:
from datasets import load_dataset
def train_bpe(
    input_path: str,
    vocab_size: int,
    # special_tokens: list[str]
) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    
    dataset = load_dataset(input_path)
    raw_text = dataset["train"]["text"][:1000]
    text = "\n".join(raw_text)

    input_byte = text.encode("utf-8")
    input_ids = list(map(int,input_byte))
    
    ##找到inputs_ids 中每一个pair 出现的次数
    def get_stats(input_ids):
        stats = {}
        
        for pair in zip(input_ids,input_ids[1:]):
            stats[pair] = stats.get(pair, 0) + 1
        return stats
    ## 找到出现最多的pair
    def find_max_pair(stats):
        max_pair = max(stats, key=stats.get)
        return max_pair
    
    ## 更新input_ids， （把找到的最多次数的pair merge 进去然后重复）
    def merge_ids(input_ids,merge_pair,merge_idx):
        
        new_ids = []
        
        i = 0
        
        while i < len(input_ids):
            
            if i < len(input_ids) - 1 and input_ids[i] == merge_pair[0] and input_ids[i+1] == merge_pair[1]:
                new_ids.append(merge_idx)
                i +=2
            else:
                new_ids.append(input_ids[i])
                i+=1
                
        return new_ids
   
   ## merge 我们需要的次数
    num_merges = vocab_size - 256
    
    merges = {}

    for i in range(num_merges):
        stats = get_stats(input_ids)
        max_pair = find_max_pair(stats)
        idx = 256+i
        input_ids = merge_ids(input_ids,merge_pair=max_pair,merge_idx=idx)
        merges[max_pair] = idx
        
    ## 构造我们需要的返回的内容
    vocab = {i:bytes([i]) for i in range(256)}
    
    merges_list = []
    for (p0,p1), idx in merges.items():
        vocab[idx] = vocab[p0] + vocab[p1]
        
        merges_list.append((vocab[p0],vocab[p1]))
    
    return (vocab,merges_list)
        
vocab, merges_list = train_bpe("roneneldan/TinyStories",400)

print(vocab)
print(merges_list)

	•	encode：字符串 → UTF-8 bytes → 反复按 merges_list 合并 → token ids
	•	decode：token ids → vocab[id]（bytes）→ 拼起来 → UTF-8 decode
    

In [ ]:
def decode(ids,vocab):
    output = b"".join(vocab[i] for i in ids).decode("utf-8", errors="replace")
    
    return output

decode([72, 101, 108, 108, 111, 97, 115, 100],vocab=vocab)

In [ ]:
def encode(input,vocab,merges_list):
    ## 先把input 变成utf8
    input_ids = [bytes([b]) for b in input.encode("utf-8")]
    ## 根据 merges list 里面的内容一点一点合并原始input ids
    for (a,b) in merges_list:
        
        i = 0
        new_ids = []
        while i < len(input_ids):
            if i < len(input_ids) -1 and input_ids[i] == a and input_ids[i+1] == b:
                new_ids.append(a+b)
                i+=2
            else:
                new_ids.append(input_ids[i])
                i+=1
        input_ids = new_ids
    
    ## 用vocab 变成数字
    vocab = {v:k for k,v in vocab.items()}
    
    input_ids = [vocab[bytes] for bytes in input_ids]
            
        
        
    return input_ids
        



In [ ]:
example_text = "Reinforcement learning (RL) has recently achieved remarkable success in eliciting visual reasoning within Multimodal Large Language Models (MLLMs). However, existing approaches typically train separate models for different tasks and treat image and video reasoning as disjoint domains. This results in limited scalability toward a multimodal reasoning generalist, which restricts practical versatility and hinders potential knowledge sharing across tasks and modalities. To this end, we propose OneThinker, an all-in-one reasoning model that unifies image and video understanding across diverse fundamental visual tasks, including question answering, captioning, spatial and temporal grounding, tracking, and segmentation. To achieve this, we construct the OneThinker-600k training corpus covering all these tasks and employ commercial models for CoT annotation, resulting in OneThinker-SFT-340k for SFT cold start. Furthermore, we propose EMA-GRPO to handle reward heterogeneity in multi-task RL by tracking task-wise moving averages of reward standard deviations for balanced optimization. Extensive experiments on diverse visual benchmarks show that OneThinker delivers strong performance on 31 benchmarks, across 10 fundamental visual understanding tasks. Moreover, it exhibits effective knowledge transfer between certain tasks and preliminary zero-shot generalization ability, marking a step toward a unified multimodal reasoning generalist. All code, model, and data are released."

raw = example_text.encode("utf-8")

print(f"raw text with utf8 encoding has a total lengh of {len(raw)}")

print("="*10)

encoded = encode(example_text,vocab,merges_list)

print(f"raw text with trained tokenizer has a total length of {len(encoded)}")

if example_text == decode(encoded,vocab):
    print("True")

In [ ]:
import regex as re

In [ ]:
pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(pat,"Hello Worssrrrtttsldsre'rere!"))

In [ ]:
import regex as re

def encode(text,pat):
    tokens = re.findall(pat, text)
    print(tokens)
    ids = []
    for tok in tokens:
        byte_ids = list(tok.encode("utf-8"))
        ids.extend(byte_ids)
    
    # 再用 merges 做 BPE merge（你已经写好了逻辑）
    # 这里略，逻辑和训练阶段一样
    
    return ids

encode("Hello World Im a student!",pat)

In [18]:
from datasets import load_dataset
dataset = load_dataset("roneneldan/TinyStories")
input_ids = dataset["train"]["text"][:500]

print(f"You are training BPE on {len(input_ids)} texts on Tiny Stroies dataset")
print("="*80)

input_ids = "".join(text for text in input_ids)

print(f"Your current training data contains {len(input_ids)} chars")

input_ids = list(input_ids.encode("utf-8"))

len(input_ids)
print(f"Your training data contains {len(input_ids)} bytes")

You are training BPE on 500 texts on Tiny Stroies dataset
Your current training data contains 402434 chars
Your training data contains 403999 bytes


In [ ]:
from datasets import load_dataset






def preprocessing(input_path:str) -> list[int]:
    
    if "TinyStories" in input_path:
        dataset = load_dataset("roneneldan/TinyStories")
    else: 
        raise ValueError("input_path need to be tiny stories!")
    
    input_ids = dataset["train"]["text"][:500]

    print(f"You are training BPE on {len(input_ids)} texts on Tiny Stroies dataset")
    print("="*80)

    input_ids = "".join(text for text in input_ids)

    print(f"Your current training data contains {len(input_ids)} chars")

    input_ids = list(input_ids.encode("utf-8"))

    len(input_ids)
    print(f"Your training data contains {len(input_ids)} bytes")
    
    return input_ids


def get_stats_on_inputs_ids(input_ids:list[int]) -> dict[tuple[int,int],int]:
    stats = {}
    
    for pair in zip(input_ids,input_ids[1:]):
        stats[pair] = stats.get(pair,0) + 1 
    
    return stats

def find_pair(stats:dict[tuple[int,int],int])->tuple[int,int]:
    
    pair = max(stats,key=stats.get)
    
    return pair

def merge(input_ids:list[int], merge_pair:tuple[int,int], idx:int):
    
    new_ids = []
    i = 0
    while i < len(input_ids):
        if i < len(input_ids) -1 and input_ids[i] == merge_pair[0] and input_ids[i+1] == merge_pair[1]:
            new_ids.append(idx)
            i+=2
        else:
            new_ids.append(input_ids[i])
            i+=1
    return new_ids 

def train_bpe(input_path: str, vocab_size: int, special_tokens: list[str]) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    
    ## 1. preprocessing input text
    input_ids = preprocessing(input_path=input_path)

    vocab = {i:bytes([i]) for i in range(256)}
    
    merges = []
     
    num_merge = vocab_size - 256
   
    
    
    
    for i in range(num_merge):
        current_stats = get_stats_on_inputs_ids(input_ids=input_ids)
        merge_pair = find_pair(current_stats)
        
        idx = i + 256
        
        print(f"merging {merge_pair} into {idx}")
        
        input_ids = merge(input_ids=input_ids,
                        merge_pair=merge_pair,
                        idx=i+256)
        
        p0, p1 = vocab[merge_pair[0]], vocab[merge_pair[1]]
        
        merges.append((p0,p1))
        vocab[idx] = vocab[merge_pair[0]] + vocab[merge_pair[1]]
        
    idx = 256 + num_merge
    for tok in special_tokens:
        vocab[idx] = tok.encode("utf-8")
        idx+=1
        
    return vocab, merges 

In [4]:
from typing import Iterable, Iterator
from train_utils import load_vocab, load_merges

class Tokenizer:
    def __init__(self, vocab, merges, special_tokens=None):
        self.vocab = vocab                      # dict[int, bytes]
        self.merges = merges                    # list[tuple[bytes, bytes]]
        self.special_tokens = special_tokens or []

        # decode 用的反向表（非常有用）
        self.id_to_bytes = vocab
        self.bytes_to_id = {v: k for k, v in vocab.items()}

    @classmethod
    def from_files(cls, vocab_path, merges_path, special_tokens=None):
        vocab = load_vocab(vocab_path)
        merges = load_merges(merges_path)
        return cls(vocab, merges, special_tokens)

    def encode(self, text: str) -> list[int]:
        raise NotImplementedError("encode() not implemented yet")

    def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]:
        for text in iterable:
            yield from self.encode(text)

    def decode(self, ids: list[int]) -> str:
        byte_seq = b"".join(self.vocab[i] for i in ids)
        return byte_seq.decode("utf-8", errors="replace")
    
tok = Tokenizer.from_files("vocab.json", "merges.txt")

print(tok.vocab[256])     # 应该是 b'th' 或类似
print(len(tok.merges))    # 应该是 vocab_size - 256

ValueError: too many values to unpack (expected 2)